In [1]:
// JAR 파일 경로를 변수로 출력한 후, 해당 경로를 매직 명령어에 사용
%jars jars/*.jar // jars 디렉토리에 있는 모든 JAR 파일 추가


In [5]:
import org.apache.spark.sql.SparkSession;
import org.apache.spark.sql.Encoders;
import org.apache.spark.sql.Dataset;
import org.apache.spark.sql.Encoder;
import org.apache.spark.sql.types.StructType;
import org.apache.spark.sql.types.DataTypes;
import java.io.Serializable;

In [6]:
import org.apache.spark.sql.Encoder;
import org.apache.commons.lang3.RandomStringUtils;
import java.io.Serializable;
import java.util.Random;
import java.util.ArrayList;
import java.util.List;

// 자바빈으로 자바 클래스 생성
public class Usage implements Serializable {
    int uid; // user id
    String uname; // username
    int usage; // usage

    public Usage() {
    }

    public Usage(int uid, String uname, int usage) {
        this.uid = uid;
        this.uname = uname;
        this.usage = usage;
    }

    // JavaBean getters and setters
    public int getUid() {
        return this.uid;
    }

    public void setUid(int uid) {
        this.uid = uid;
    }

    public String getUname() {
        return this.uname;
    }

    public void setUname(String uname) {
        this.uname = uname;
    }

    public int getUsage() {
        return this.usage;
    }

    public void setUsage(int usage) {
        this.usage = usage;
    }

    @Override
    public String toString() {
        return "uid: " + this.uid + " uname: " + this.uname + " usage: " + this.usage;
    }
}

// 명시적 인코더 생성
Encoder<Usage> usageEncoder = Encoders.bean(Usage.class);
Random rand = new Random();
rand.setSeed(42);
List<Usage> data = new ArrayList<Usage>();

// 자바 Usage 클래스의 1000개 인스턴스 생성
for (int i = 0; i < 1000; i++) {
    data.add(new Usage(i, "user" + RandomStringUtils.randomAlphanumeric(5), rand.nextInt(1000)));
}

// Usage 형태의 데이터세트 생성
Dataset<Usage> dsUsage = spark.createDataset(data, usageEncoder);


CompilationException: 

In [19]:
import org.apache.spark.sql.Row;
import org.apache.spark.sql.RowFactory;
import org.apache.spark.sql.Encoders;
import org.apache.spark.sql.Dataset;
import org.apache.spark.sql.catalyst.encoders.RowEncoder;
import org.apache.spark.sql.types.DataTypes;
import org.apache.spark.sql.types.StructField;
import org.apache.spark.sql.types.StructType;
import org.apache.spark.api.java.function.MapFunction;

In [39]:
// src/main/java/UsageExample.java
import org.apache.spark.sql.SparkSession;
import org.apache.spark.sql.Encoders;
import org.apache.spark.sql.Dataset;
import org.apache.spark.api.java.function.MapFunction;
import org.apache.commons.lang3.RandomStringUtils;
import org.apache.spark.api.java.function.FilterFunction;

import java.io.Serializable;
import java.util.Random;
import java.util.ArrayList;
import java.util.List;

public class Usage implements Serializable {
    int uid; // user id
    String uname; // username
    int usage; // usage

    public Usage() {
    }

    public Usage(int uid, String uname, int usage) {
        this.uid = uid;
        this.uname = uname;
        this.usage = usage;
    }

    // JavaBean getters and setters
    public int getUid() {
        return this.uid;
    }

    public void setUid(int uid) {
        this.uid = uid;
    }

    public String getUname() {
        return this.uname;
    }

    public void setUname(String uname) {
        this.uname = uname;
    }

    public int getUsage() {
        return this.usage;
    }

    public void setUsage(int usage) {
        this.usage = usage;
    }

    @Override
    public String toString() {
        return "uid: " + this.uid + " uname: " + this.uname + " usage: " + this.usage;
    }

    public static Dataset<Usage> createUsageDataset(SparkSession spark) {
        // 명시적 인코더 생성
        Encoder<Usage> usageEncoder = Encoders.bean(Usage.class);
        Random rand = new Random();
        rand.setSeed(42);
        List<Usage> data = new ArrayList<Usage>();

        // 자바 Usage 클래스의 1000개 인스턴스 생성
        for (int i = 0; i < 1000; i++) {
            data.add(new Usage(i, "user" + RandomStringUtils.randomAlphanumeric(5), rand.nextInt(1000)));
        }

        // Usage 형태의 데이터세트 생성
        return spark.createDataset(data, usageEncoder);
    }
    public static double computeCostUsage(int usage) {
        if (usage > 750) {
            return (usage - 750) * 0.1 + 750 * 0.5;
        } else {
            return usage * 0.5;
        }
    }
}

public class UsageExample {
    public static void main(String[] args) {
        // 스파크 세션 생성
        SparkSession spark = SparkSession.builder()
                .appName("Usage Example")
                .master("local[*]")
                .getOrCreate();

        // Usage 데이터셋 생성
        Dataset<Usage> dsUsage = Usage.createUsageDataset(spark);
        dsUsage.show(5);

        StructType schema = DataTypes.createStructType(new StructField[]{
    DataTypes.createStructField("user_id", DataTypes.IntegerType, false),
    DataTypes.createStructField("username", DataTypes.StringType, false),
    DataTypes.createStructField("cost", DataTypes.DoubleType, false)
});


        // 인라인 MapFunction 정의
     Dataset<Double> transformedUsage = dsUsage
            .filter((FilterFunction<Usage>) u -> u.getUsage() >= 500)
            .map((MapFunction<Usage, Double>) u -> {
                if (u.getUsage() > 750) {
                    return (u.getUsage()-750) * 0.1 + 750 * 0.5;
                } else {
                    return u.getUsage() * 0.50;
                }
            }, Encoders.DOUBLE())
            .map((MapFunction<Double, Double>) value -> value + 100
                 , Encoders.DOUBLE());




        // 결과를 출력
        transformedUsage.show(5);

        // 스파크 세션 종료
        spark.stop();
    }
}


In [40]:
UsageExample.main(new String[0])

24/05/27 19:22:50 INFO SparkContext: Running Spark version 3.5.1
24/05/27 19:22:50 INFO SparkContext: OS info Windows 11, 10.0, amd64
24/05/27 19:22:50 INFO SparkContext: Java version 11.0.20
24/05/27 19:22:50 INFO ResourceUtils: ==============================================================
24/05/27 19:22:50 INFO ResourceUtils: No custom resources configured for spark.driver.
24/05/27 19:22:50 INFO ResourceUtils: ==============================================================
24/05/27 19:22:50 INFO SparkContext: Submitted application: Usage Example
24/05/27 19:22:50 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
24/05/27 19:22:50 INFO ResourceProfile: Limiting resource is cpu
24/05/27 19:22:50 INFO ResourceProfileManager: Added Resource

+---+---------+-----+
|uid|    uname|usage|
+---+---------+-----+
|  0|userYYfsS|  130|
|  1|userczNjY|  763|
|  2|useryE5ks|  248|
|  3|user8d4SJ|  884|
|  4|userFZgjR|  970|
+---+---------+-----+
only showing top 5 rows



24/05/27 19:22:51 INFO SparkContext: Starting job: show at $JShell$36Q.java:71
24/05/27 19:22:51 INFO DAGScheduler: Got job 0 (show at $JShell$36Q.java:71) with 1 output partitions
24/05/27 19:22:51 INFO DAGScheduler: Final stage: ResultStage 0 (show at $JShell$36Q.java:71)
24/05/27 19:22:51 INFO DAGScheduler: Parents of final stage: List()
24/05/27 19:22:51 INFO DAGScheduler: Missing parents: List()
24/05/27 19:22:51 INFO DAGScheduler: Submitting ResultStage 0 (MapPartitionsRDD[2] at show at $JShell$36Q.java:71), which has no missing parents
24/05/27 19:22:51 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 13.7 KiB, free 1018.8 MiB)
24/05/27 19:22:51 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 5.6 KiB, free 1018.8 MiB)
24/05/27 19:22:51 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on localhost:55942 (size: 5.6 KiB, free: 1018.8 MiB)
24/05/27 19:22:51 INFO SparkContext: Created broadcast 0 from broadca

+-----+
|value|
+-----+
|476.3|
|488.4|
|497.0|
|362.5|
|352.5|
+-----+
only showing top 5 rows



24/05/27 19:22:51 INFO SparkContext: SparkContext is stopping with exitCode 0.
24/05/27 19:22:51 INFO SparkUI: Stopped Spark web UI at http://localhost:4040
24/05/27 19:22:51 INFO MapOutputTrackerMasterEndpoint: MapOutputTrackerMasterEndpoint stopped!
24/05/27 19:22:51 INFO MemoryStore: MemoryStore cleared
24/05/27 19:22:51 INFO BlockManager: BlockManager stopped
24/05/27 19:22:51 INFO BlockManagerMaster: BlockManagerMaster stopped
24/05/27 19:22:51 INFO OutputCommitCoordinator$OutputCommitCoordinatorEndpoint: OutputCommitCoordinator stopped!
24/05/27 19:22:51 INFO SparkContext: Successfully stopped SparkContext


In [ ]:
Dataset<Double> transformedUsage = dsUsage
            .filter((FilterFunction<Usage>) u -> u.getUsage() >= 500)
            .map((MapFunction<Usage, Double>) u -> {
                if (u.getUsage() > 750) {
                    return (u.getUsage()-750) * 0.1 + 750 * 0.5;
                } else {
                    return u.getUsage() * 0.50;
                }
            }, Encoders.DOUBLE());

In [38]:
Dataset<Double> transformedUsage = dsUsage
            .filter((FilterFunction<Usage>) u -> u.getUsage() >= 500)
            .map((MapFunction<Usage, Double>) u -> {
                if (u.getUsage() > 750) {
                    return (u.getUsage()-750) * 0.1 + 750 * 0.5;
                } else {
                    return u.getUsage() * 0.50;
                }
            }, Encoders.DOUBLE())
            .map((MapFunction<Double, Double>) value -> value + 100
                 , Encoders.DOUBLE());

CompilationException: 